In [1]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Create embeddings and a vector db out of stored index
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Meilisearch

embeddings = OpenAIEmbeddings()
vectordb = Meilisearch(embedding=embeddings, index_name='elderan')

In [3]:
# Create a retriever from the vector db
retriever = vectordb.as_retriever()

query = "Who was the lord of House Arion?"
retriever.get_relevant_documents(query=query)

[Document(page_content='At the heart of it all was House Arion, ruling from their majestic fortress atop the Frostpeak', metadata={'source': 'story_1.txt'}),
 Document(page_content='atop the Frostpeak Mountains. Led by Lord Cedric Arion, a cunning and shrewd strategist, the House', metadata={'source': 'story_1.txt'}),
 Document(page_content='Amidst the turmoil, a bastard son of House Arion named Eamon Snow, skilled with a blade and', metadata={'source': 'story_1.txt'}),
 Document(page_content='could unravel a web of lies. A fallen knight named Ser Jorran sought redemption in the face of', metadata={'source': 'story_1.txt'})]

In [4]:
# Create a conversation agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, max_tokens=500, )
conversational_memory = ConversationBufferWindowMemory(memory_key='chat_history', k=5, return_messages=True)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

In [5]:
# Test the retriever
qa.run(query)

'The lord of House Arion was Lord Cedric Arion.'

In [6]:
# Let's convert the retrieval chain into a tool
from langchain.agents import Tool

tools = [
    Tool(
        name='QuipBot Knowledge Base', 
        func=qa.run, 
        description='use this tool when answering general knowledge queries to get more information about the topic'
    )
]

In [7]:
# Now, we can initialize the agent
from langchain.agents import initialize_agent
from langchain.agents import AgentType

system_message = "You are Elderan, an intelligent AI assistant who will help people to know more about the Kingdom of Eldoria. Answer questions that are asked to you. If you don't have proper and enough information politely say you don't know."

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
    agent_kwargs={
             "system_message": system_message
            }
)


In [8]:
agent.run('Who was the lord of House Arion?')



> Entering new AgentExecutor chain...
{
    "action": "QuipBot Knowledge Base",
    "action_input": "Who was the lord of House Arion?"
}
Observation: The lord of House Arion was Lord Cedric Arion.
Thought:{
    "action": "Final Answer",
    "action_input": "The lord of House Arion was Lord Cedric Arion."
}

> Finished chain.


'The lord of House Arion was Lord Cedric Arion.'

In [9]:
agent.run('Does he have any son?')



> Entering new AgentExecutor chain...
{
    "action": "QuipBot Knowledge Base",
    "action_input": "Does Lord Cedric Arion have any son?"
}
Observation: Based on the given context, it is not mentioned whether Lord Cedric Arion has any sons.
Thought:{
    "action": "Final Answer",
    "action_input": "It is not mentioned whether Lord Cedric Arion has any sons."
}

> Finished chain.


'It is not mentioned whether Lord Cedric Arion has any sons.'

In [10]:
agent.run('Who is Eamon Snow?')



> Entering new AgentExecutor chain...
{
    "action": "QuipBot Knowledge Base",
    "action_input": "Who is Eamon Snow?"
}
Observation: Eamon Snow is a skilled swordsman and a bastard son of House Arion. He plays a role in the turmoil and conflicts within the realm of Eldoria.
Thought:{
    "action": "Final Answer",
    "action_input": "Eamon Snow is a skilled swordsman and a bastard son of House Arion. He plays a role in the turmoil and conflicts within the realm of Eldoria."
}

> Finished chain.


'Eamon Snow is a skilled swordsman and a bastard son of House Arion. He plays a role in the turmoil and conflicts within the realm of Eldoria.'

In [12]:
agent.run('Where does his loyalty lie?')



> Entering new AgentExecutor chain...
{
    "action": "QuipBot Knowledge Base",
    "action_input": "Eamon Snow loyalty"
}
Observation: Eamon Snow's loyalty is initially towards his House, House Arion. However, as the brewing storm threatens to shatter the realm, it is unclear where his loyalty will ultimately lie.
Thought:{
    "action": "Final Answer",
    "action_input": "Eamon Snow's loyalty is initially towards his House, House Arion. However, as the brewing storm threatens to shatter the realm, it is unclear where his loyalty will ultimately lie."
}

> Finished chain.


"Eamon Snow's loyalty is initially towards his House, House Arion. However, as the brewing storm threatens to shatter the realm, it is unclear where his loyalty will ultimately lie."

In [13]:
agent.run('Who are you?')



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I am Elderan, an intelligent AI assistant here to help you learn more about the Kingdom of Eldoria."
}

> Finished chain.


'I am Elderan, an intelligent AI assistant here to help you learn more about the Kingdom of Eldoria.'